In [4]:
import ipaddress

net = ipaddress.ip_network('123.45.67.64/27')

for a in net:
    print(a)

123.45.67.64
123.45.67.65
123.45.67.66
123.45.67.67
123.45.67.68
123.45.67.69
123.45.67.70
123.45.67.71
123.45.67.72
123.45.67.73
123.45.67.74
123.45.67.75
123.45.67.76
123.45.67.77
123.45.67.78
123.45.67.79
123.45.67.80
123.45.67.81
123.45.67.82
123.45.67.83
123.45.67.84
123.45.67.85
123.45.67.86
123.45.67.87
123.45.67.88
123.45.67.89
123.45.67.90
123.45.67.91
123.45.67.92
123.45.67.93
123.45.67.94
123.45.67.95


In [5]:
net6 = ipaddress.ip_network('12:3456:78:90ab:cd:ef01:23:30/125')

for a in net6:
    print(a)

12:3456:78:90ab:cd:ef01:23:30
12:3456:78:90ab:cd:ef01:23:31
12:3456:78:90ab:cd:ef01:23:32
12:3456:78:90ab:cd:ef01:23:33
12:3456:78:90ab:cd:ef01:23:34
12:3456:78:90ab:cd:ef01:23:35
12:3456:78:90ab:cd:ef01:23:36
12:3456:78:90ab:cd:ef01:23:37


In [8]:
print(net.num_addresses)
print(net[0])
print(net[-1])

a = ipaddress.ip_address('123.45.67.69')
print(a in net)

32
123.45.67.64
123.45.67.95
True


In [9]:
inet = ipaddress.ip_interface('123.45.67.73/27')
print(inet.network)
print(inet.ip)

123.45.67.64/27
123.45.67.73
